# Example 3: Extract Class, Move Field, Move Method

[Extract class in the refactoring catalog.](http://refactoring.com/catalog/extractClass.html)

[Move field in the refactoring catalog](http://refactoring.com/catalog/moveField.html).

[Move method in the refactoring catalog](http://refactoring.com/catalog/moveMethod.html).

- Do this when a class is responsible for too many things (> 5 methods, > 5 properties)
- Define a new class with all the fields you need to do the work.
- Implement `@property` and `@property.setter` to point at the new helper class so old usage continues to work.
- Move the methods that interact with these properties onto the interior class, instead of on the exterior class. Use `@property` to point existing usage at the new helper class.
- Use warnings to track down old usage. Move that usage to directly interact with the interior class. Remove all the `@property`s you used for indirection.

In [107]:
class Pet:
    def __init__(self, name):
        self.name = name

In [108]:
my_pet = Pet('Gregory the Gila Monster')
print(my_pet.name)

Gregory the Gila Monster


---

More complex

In [109]:
class Pet:
    def __init__(self, name, age):
        self.name = name
        self.age = age

In [110]:
my_pet = Pet('Gregory the Gila Monster', 3)
print('%s is %d years old' % (my_pet.name, my_pet.age))

Gregory the Gila Monster is 3 years old


Add an action method for feeding the pet.

In [111]:
class Pet:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        self.treats_eaten = 0
    
    def give_treats(self, count):
        self.treats_eaten += count

In [113]:
my_pet = Pet('Gregory the Gila Monster', 3)
my_pet.give_treats(2)
print('%s ate %d treats' % (my_pet.name, my_pet.treats_eaten))

Gregory the Gila Monster ate 2 treats


---

Now we want to know what kind of pet it is, so we add various attributes and helper methods.

In [118]:
class Pet:
    def __init__(self, name, age, *, has_scales=False, lays_eggs=False, drinks_milk=False):
        self.name = name
        self.age = age
        self.treats_eaten = 0
        self.has_scales = has_scales
        self.lays_eggs = lays_eggs
        self.drinks_milk = drinks_milk

    def give_treats(self, count):
        self.treats_eaten += count
    
    @property
    def needs_heat_lamp(self):
        return (
            self.has_scales and
            self.lays_eggs and
            not self.drinks_milk)

In [119]:
my_pet = Pet('Gregory the Gila Monster', 3, has_scales=True, lays_eggs=True)
print('%s needs a heat lamp? %s' % (my_pet.name, my_pet.needs_heat_lamp))

Gregory the Gila Monster needs a heat lamp? True


In [120]:
my_pet.give_treats(3)
my_pet.give_treats(2)
print('%s ate %d treats' % (my_pet.name, my_pet.treats_eaten))

Gregory the Gila Monster ate 5 treats


---

Maybe this class has too many responsibilities. We should separate the details of what the animal is and its intrinsic attributes from what we know about the pet.

In [103]:
import warnings

In [125]:
class Animal:
    def __init__(self, *, has_scales=False, lays_eggs=False, drinks_milk=False):
        self.has_scales = has_scales
        self.lays_eggs = lays_eggs
        self.drinks_milk = drinks_milk

class Pet:
    def __init__(self, name, age, animal=None, **kwargs):
        self.name = name
        self.age = age
        self.treats_eaten = 0

        if kwargs and animal is not None:
            raise TypeError('Must supply either an Animal instance or keyword arguments')

        if kwargs:
            warnings.warn('Must directly pass an Animal instance', DeprecationWarning)
            animal = Animal(**kwargs)
        elif animal is None:
            raise TypeError('Must supply "animal"')
        
        self.animal = animal

    def give_treats(self, count):
        self.treats_eaten += count

    @property
    def needs_heat_lamp(self):  # Changed to reference self.animal
        return (
            self.animal.has_scales and
            self.animal.lays_eggs and
            not self.animal.drinks_milk)

    @property
    def has_scales(self):
        warnings.warn('Use animal.has_scales instead', DeprecationWarning)
        return self.animal.has_scales

    @property
    def lays_eggs(self):
        warnings.warn('Use animal.lays_eggs instead', DeprecationWarning)
        return self.animal.lays_eggs

    @property
    def drinks_milk(self):
        warnings.warn('Use animal.drinks_milk instead', DeprecationWarning)
        return self.animal.drinks_milk

---

Old usage keeps working

In [126]:
my_pet = Pet('Gregory the Gila Monster', 3, has_scales=True, lays_eggs=True)  # Warning expected
print('%s needs a heat lamp? %s' % (my_pet.name, my_pet.needs_heat_lamp))

Gregory the Gila Monster needs a heat lamp? True


/Users/bslatkin/projects/pycon2016/lib/python3.5/site-packages/ipykernel/__main__.py:17: DeprecationWarning: Must directly pass an Animal instance


---

Missing parameters for new usage will break like you called the function incorrectly.

In [127]:
my_pet = Pet('Gregory the Gila Monster', 3)  # Error expected

TypeError: Must supply "animal"

Providing both types of usage will fail.

In [128]:
my_pet = Pet('Gregory the Gila Monster', 3, animal, has_scales=True)  # Error expected

TypeError: Must supply either an Animal instance or keyword arguments

---

New usage works and no warnings.

In [130]:
animal = Animal(has_scales=True, lays_eggs=True)
my_pet = Pet('Gregory the Gila Monster', 3, animal)
print('%s needs a heat lamp? %s' % (my_pet.name, my_pet.needs_heat_lamp))

Gregory the Gila Monster needs a heat lamp? True


---

Now assume that all of the references have been moved to the new version.

In [70]:
class Animal:
    def __init__(self, *, has_scales=False, lays_eggs=False, drinks_milk=False):
        self.has_scales = has_scales
        self.lays_eggs = lays_eggs
        self.drinks_milk = drinks_milk

class Pet:
    def __init__(self, name, age, animal):
        self.name = name
        self.age = age
        self.animal = animal
        self.treats_eaten = 0

    def give_treats(self, count):
        self.treats_eaten += count

    @property
    def needs_heat_lamp(self):
        return (
            self.animal.has_scales and
            self.animal.lays_eggs and
            not self.animal.drinks_milk)

In [131]:
animal = Animal(has_scales=True, lays_eggs=True)
my_pet = Pet('Gregory the Gila Monster', 3, animal)
print('%s needs a heat lamp? %s' % (my_pet.name, my_pet.needs_heat_lamp))

Gregory the Gila Monster needs a heat lamp? True


---

Now we want to move the `needs_heat_lamp` method into the `Animal` class. It's best not to refactor too many things at the same time. The point here is you can do it piecemeal and you'll existing a transitional state where part of your APIs looks good and part of it doesn't look good yet.

In [132]:
class Animal:
    def __init__(self, *, has_scales=False, lays_eggs=False, drinks_milk=False):
        self.has_scales = has_scales
        self.lays_eggs = lays_eggs
        self.drinks_milk = drinks_milk

    @property
    def needs_heat_lamp(self):
        return (
            self.has_scales and
            self.lays_eggs and
            not self.drinks_milk)
        
class Pet:
    def __init__(self, name, age, animal):
        self.name = name
        self.age = age
        self.animal = animal
        self.treats_eaten = 0

    def give_treats(self, count):
        self.treats_eaten += count

    @property
    def needs_heat_lamp(self):
        warnings.warn('Must use animal.needs_heat_lamp', DeprecationWarning)
        return self.animal.needs_heat_lamp

In [133]:
animal = Animal(has_scales=True, lays_eggs=True)
my_pet = Pet('Gregory the Gila Monster', 3, animal)
print('%s needs a heat lamp? %s' % (my_pet.name, my_pet.needs_heat_lamp))  # Warning expected

Gregory the Gila Monster needs a heat lamp? True


/Users/bslatkin/projects/pycon2016/lib/python3.5/site-packages/ipykernel/__main__.py:26: DeprecationWarning: Must use animal.needs_heat_lamp


Fix all of the usages

In [145]:
class Animal:
    def __init__(self, *, has_scales=False, lays_eggs=False, drinks_milk=False):
        self.has_scales = has_scales
        self.lays_eggs = lays_eggs
        self.drinks_milk = drinks_milk

    @property
    def needs_heat_lamp(self):
        return (
            self.has_scales and
            self.lays_eggs and
            not self.drinks_milk)
        
class Pet:
    def __init__(self, name, age, animal):
        self.name = name
        self.age = age
        self.animal = animal
        self.treats_eaten = 0

    def give_treats(self, count):
        self.treats_eaten += count

Old usage stops working

In [146]:
animal = Animal(has_scales=True, lays_eggs=True)
my_pet = Pet('Gregory the Gila Monster', 3, animal)
print('%s needs a heat lamp? %s' % (my_pet.name, my_pet.needs_heat_lamp))  # Error expected

AttributeError: 'Pet' object has no attribute 'needs_heat_lamp'

New usage works without any problems.

In [147]:
animal = Animal(has_scales=True, lays_eggs=True)
my_pet = Pet('Gregory the Gila Monster', 3, animal)
print('%s needs a heat lamp? %s' % (my_pet.name, my_pet.animal.needs_heat_lamp))

Gregory the Gila Monster needs a heat lamp? True


This same approach works for both methods and `@property` methods.

---

Another intrinsic property of the pet is its `age`. We should refactor this into the interior class. This is more challenging because it's a property being assigned. If you do it the same was as the other attributes, it will break.

In [165]:
animal = Animal(has_scales=True, lays_eggs=True)
my_pet = Pet('Gregory the Gila Monster', 3, animal)
my_pet.age = 5
print('%s is %d years old' % (my_pet.name, my_pet.age))

TypeError: __init__() missing 1 required positional argument: 'age'

In [166]:
class Animal:
    def __init__(self, age=None, *, has_scales=False, lays_eggs=False, drinks_milk=False):
        if age is None:
            warnings.warn('Should specify "age" for Animal')
        self.age = age
        self.has_scales = has_scales
        self.lays_eggs = lays_eggs
        self.drinks_milk = drinks_milk

    @property
    def needs_heat_lamp(self):
        return (
            self.has_scales and
            self.lays_eggs and
            not self.drinks_milk)
        
class Pet:
    def __init__(self, name, age_or_animal, maybe_animal=None):
        self.name = name

        if maybe_animal is not None:
            warnings.warn('Should specify "age" for Animal')
            self.animal = maybe_animal
            self.animal.age = age_or_animal
        else:
            self.animal = age_or_animal

        self.treats_eaten = 0

    def give_treats(self, count):
        self.treats_eaten += count

    @property
    def age(self):
        warnings.warn('Should use animal.age')
        return self.animal.age

In [168]:
animal = Animal(has_scales=True, lays_eggs=True)  # Warning expected
my_pet = Pet('Gregory the Gila Monster', 3, animal)  # Warning expected
print('%s is %d years old' % (my_pet.name, my_pet.age))  # Warning expected

Gregory the Gila Monster is 3 years old


/Users/bslatkin/projects/pycon2016/lib/python3.5/site-packages/ipykernel/__main__.py:4: UserWarning: Should specify "age" for Animal
/Users/bslatkin/projects/pycon2016/lib/python3.5/site-packages/ipykernel/__main__.py:22: UserWarning: Should specify "age" for Animal
/Users/bslatkin/projects/pycon2016/lib/python3.5/site-packages/ipykernel/__main__.py:35: UserWarning: Should use animal.age


New usage has no warnings.

In [169]:
animal = Animal(3, has_scales=True, lays_eggs=True)
my_pet = Pet('Gregory the Gila Monster', animal)
print('%s is %d years old' % (my_pet.name, my_pet.animal.age))

Gregory the Gila Monster is 3 years old


The middle state breaks when you try to assign to the age attribute.

In [170]:
my_pet.age = 5  # Error expected

AttributeError: can't set attribute

What's missing is the `@property.setter` that can properly handle assignment during the transition period.

In [179]:
class Animal:
    def __init__(self, age=None, *, has_scales=False, lays_eggs=False, drinks_milk=False):
        if age is None:
            warnings.warn('Should specify "age" for Animal')
        self.age = age
        self.has_scales = has_scales
        self.lays_eggs = lays_eggs
        self.drinks_milk = drinks_milk

    @property
    def needs_heat_lamp(self):
        return (
            self.has_scales and
            self.lays_eggs and
            not self.drinks_milk)
        
class Pet:
    def __init__(self, name, age_or_animal, maybe_animal=None):
        self.name = name

        if maybe_animal is not None:
            warnings.warn('Should specify "age" for Animal')
            self.animal = maybe_animal
            self.animal.age = age_or_animal
        else:
            self.animal = age_or_animal

        self.treats_eaten = 0

    def give_treats(self, count):
        self.treats_eaten += count

    @property
    def age(self):
        warnings.warn('Should use animal.age')
        return self.animal.age

    @age.setter
    def age(self, new_age):
        warnings.warn('Should assign animal.age')
        self.animal.age = new_age

In [180]:
animal = Animal(3, has_scales=True, lays_eggs=True)
my_pet = Pet('Gregory the Gila Monster', animal)
my_pet.age = 5  # Warning expected

/Users/bslatkin/projects/pycon2016/lib/python3.5/site-packages/ipykernel/__main__.py:40: UserWarning: Should assign animal.age


Now we can move everything over to the new usage, and remove the old way.

In [183]:
class Animal:
    def __init__(self, age, *, has_scales=False, lays_eggs=False, drinks_milk=False):
        self.age = age
        self.has_scales = has_scales
        self.lays_eggs = lays_eggs
        self.drinks_milk = drinks_milk

    @property
    def needs_heat_lamp(self):
        return (
            self.has_scales and
            self.lays_eggs and
            not self.drinks_milk)
        
class Pet:
    def __init__(self, name, animal):
        self.name = name
        self.animal = animal
        self.treats_eaten = 0

    def give_treats(self, count):
        self.treats_eaten += count

In [185]:
animal = Animal(3, has_scales=True, lays_eggs=True)
my_pet = Pet('Gregory the Gila Monster', animal)
my_pet.animal.age = 5
print('%s is %d years old' % (my_pet.name, my_pet.animal.age))

Gregory the Gila Monster is 5 years old


---

One gotcha is you can still accidentally use the old assignments because Python will add an attribute to a class when it's assigned.

In [188]:
animal = Animal(3, has_scales=True, lays_eggs=True)
my_pet = Pet('Gregory the Gila Monster', animal)
my_pet.age = 5  # Old usage doesn't raise an error
print('%s is %d years old' % (my_pet.name, my_pet.animal.age))  # Prints wrong result

Gregory the Gila Monster is 3 years old


So you can be extra paranoid for old usage a leave a tombstone property to prevent this. Old habits and muscle memory die hard.

In [189]:
class Animal:
    def __init__(self, age, *, has_scales=False, lays_eggs=False, drinks_milk=False):
        self.age = age
        self.has_scales = has_scales
        self.lays_eggs = lays_eggs
        self.drinks_milk = drinks_milk

    @property
    def needs_heat_lamp(self):
        return (
            self.has_scales and
            self.lays_eggs and
            not self.drinks_milk)
        
class Pet:
    def __init__(self, name, animal):
        self.name = name
        self.animal = animal
        self.treats_eaten = 0

    def give_treats(self, count):
        self.treats_eaten += count

    @property
    def age(self):
        raise AttributeError('Must use animal.age')

    @age.setter
    def age(self, new_age):
        raise AttributeError('Must assign animal.age')

Now we'll get an error immediately on accidental usage of the pre-migration approach.

In [191]:
animal = Animal(3, has_scales=True, lays_eggs=True)
my_pet = Pet('Gregory the Gila Monster', animal)
my_pet.age = 5  # Old usage raises an error

AttributeError: Must assign animal.age